In [1]:
import sys
import numpy as np
import matplotlib.cm as cm
import plotly.plotly as py
from plotly.graph_objs import *


ImportError: No module named 'plotly'

In [ ]:
def read_off(filename):
    with open(filename) as fn:
        line = fn.readline()
        if (''.join(line.split()) != 'OFF'):
            print(filename + ' is not an OFF file')
            sys.exit(0)
        
        line = fn.readline()
        nvfe = np.asarray(line.split(),dtype=int)
        vertices = np.zeros((nvfe[0],3))
        triangles = np.zeros((nvfe[1],3),dtype=int)
        for i in range(nvfe[0]):
            line = fn.readline()
            vertices[i] = np.asarray(line.split(),dtype=float)
            
        for i in range(nvfe[1]):
            line = fn.readline()
            triangles[i] = np.asarray(line.split(),dtype=int)[1:]
            
        return(vertices,triangles)

In [ ]:
def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value zval to a corresponding color in the colormap
    
    if vmin>vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+\
           ','+'{:d}'.format(int(B*255+0.5))+')'   

def tri_indices(simplices):
    #simplices is a numpy array defining the simplices of the triangularization
    #returns the lists of indices i, j, k
    
    return ([triplet[c] for triplet in simplices] for c in range(3))

def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data
    
    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)# vertices of the surface triangles     
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)  
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean] 
    I,J,K=tri_indices(simplices)
    
    triangles=Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor, 
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )
    
    if plot_edges is None:# the triangle sides are not plotted 
        return Data([triangles])
    

In [ ]:
lv,lt = read_off('socket.off')

In [ ]:
data1=plotly_trisurf(lv[:,0],lv[:,0],lv[:,0], lt, colormap=cm.RdBu, plot_edges=None)

In [ ]:
axis = dict(
showbackground=True, 
backgroundcolor="rgb(230, 230,230)",
gridcolor="rgb(255, 255, 255)",      
zerolinecolor="rgb(255, 255, 255)",  
    )

layout = Layout(
         title='Moebius band triangulation',
         width=800,
         height=800,
         scene=Scene(  
         xaxis=XAxis(axis),
         yaxis=YAxis(axis), 
         zaxis=ZAxis(axis), 
        aspectratio=dict(
            x=1,
            y=1,
            z=0.5
        ),
        )
        )
fig1 = Figure(data=data1,layout=layout)

py.iplot(fig1, filename='Moebius-band-trisurf')